<h1 align='center'>Nova Mart Data Analysis Project</h1>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

<h2 style="color:purple" align="center">Data Import</h2>

#### Read Data From CSV Files

In [2]:
df_campaigns = pd.read_csv("datasets/dim_campaigns.csv")
df_products = pd.read_csv("datasets/dim_products.csv")
df_stores = pd.read_csv("datasets/dim_stores.csv")
df_events = pd.read_csv("datasets/fact_events.csv")

In [3]:
df_campaigns.head()

,campaign_id,campaign_name,start_date,end_date
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023
1,CAMP_SAN_01,Sankranti,10-01-2024,16-01-2024


In [4]:
df_products.head(3)

,product_code,product_name,category
0,P01,Atliq_Masoor_Dal (1KG),Grocery & Staples
1,P02,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples
2,P03,Atliq_Suflower_Oil (1L),Grocery & Staples


In [5]:
df_stores.head(3)

,store_id,city
0,STTRV-0,Trivandrum
1,STMDU-3,Madurai
2,STHYD-6,Hyderabad


In [6]:
df_events.head(3)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488


#### Client Requests
*1. The operations team wants to ensure the integrity of the events data by removing duplicates. Check for and remove duplicate rows in the events dataframe based on store_id, campaign_id, and product_code. How many duplicate rows were removed?*

In [7]:
print(f"Number of rows and columns in events dataframe: {df_events.shape}")
df_events.shape

Number of rows and columns in events dataframe: (1500, 9)


(1500, 9)

In [8]:
# 1. Removing duplicates based on store_id, campaign_id, and product_code
# Check the initial row count
initial_row_count = len(df_events)
initial_row_count

1500

In [9]:
df_events['store_id'].nunique()

50

In [10]:
df_events['campaign_id'].nunique()

2

In [11]:
df_events['product_code'].nunique()

15

In [12]:
df_events[['store_id', 'campaign_id', 'product_code']].nunique()

store_id        50
campaign_id      2
product_code    15
dtype: int64

In [13]:
df_events[['store_id', 'campaign_id', 'product_code']].apply(tuple, axis=1).nunique()

1500

In [14]:
df_events[df_events.duplicated(subset=['store_id', 'campaign_id', 'product_code'], keep=False)]

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)


In [15]:
df_events[df_events.duplicated(subset=['store_id', 'campaign_id', 'product_code'], keep=False)].shape

(0, 9)

In [16]:
df_events_1 = df_events.drop_duplicates(subset=['store_id', 'campaign_id', 'product_code'])
df_events_1.shape

(1500, 9)

In [17]:
final_rows_count = len(df_events_1)
final_rows_count

1500

In [18]:
duplicated_rows = initial_row_count - final_rows_count
duplicated_rows

0

#### Client Requests
*2. How many cities have more than 5 stores?*


In [19]:
print(f"Number of rows and columns in events dataframe: {df_stores.shape}")
df_stores.shape

Number of rows and columns in events dataframe: (50, 2)


(50, 2)

In [20]:
df_stores.head(4)

,store_id,city
0,STTRV-0,Trivandrum
1,STMDU-3,Madurai
2,STHYD-6,Hyderabad
3,STVSK-1,Visakhapatnam


In [21]:
# 2. Counting cities with more than 5 stores
# Group by city and count the unique store
store_counts = df_stores.groupby('city')['store_id']
store_counts

In [22]:
store_counts = df_stores.groupby('city')['store_id'].nunique()
store_counts

city
Bengaluru        10
Chennai           8
Coimbatore        5
Hyderabad         7
Madurai           4
Mangalore         3
Mysuru            4
Trivandrum        2
Vijayawada        2
Visakhapatnam     5
Name: store_id, dtype: int64

In [23]:
store_counts = pd.DataFrame(df_stores.groupby('city')['store_id'].nunique())
store_counts.rename(columns={'store_id':'unique_store_count'}, inplace=True)
store_counts

,unique_store_count
city,
Bengaluru,10
Chennai,8
Coimbatore,5
Hyderabad,7
Madurai,4
Mangalore,3
Mysuru,4
Trivandrum,2
Vijayawada,2


In [24]:
store_counts[store_counts.unique_store_count > 5]

,unique_store_count
city,
Bengaluru,10
Chennai,8
Hyderabad,7


In [25]:
cities_with_more_than_5_stores = store_counts[store_counts.unique_store_count > 5] 
cities_with_more_than_5_stores

,unique_store_count
city,
Bengaluru,10
Chennai,8
Hyderabad,7


In [26]:
num_cities = cities_with_more_than_5_stores.shape[0]
print(f"The number of cities with more than 5 stores is: {num_cities}")
num_cities

The number of cities with more than 5 stores is: 3


3

#### Client Requests
*3.  The sales team has identified missing values in the quantity_sold(before_promo) data. Estimate these values using the median quantity sold before the promotion. How many missing values were filled, and what is the median used for imputation?*

In [27]:
df_events_1.isnull().sum()

event_id                       0
store_id                       0
campaign_id                    0
product_code                   0
base_price(before_promo)       0
quantity_sold(before_promo)    0
promo_type                     0
base_price(after_promo)        0
quantity_sold(after_promo)     0
dtype: int64

In [28]:
median_value = df_events['quantity_sold(before_promo)'].median()
median_value

np.float64(78.0)

In [29]:
missing_values = df_events_1['quantity_sold(before_promo)'].isnull().sum()
print(f"Number of missing values: {missing_values}")

Number of missing values: 0


In [30]:
df_events_1['quantity_sold(before_promo)'].mean(), df_events_1['quantity_sold(before_promo)'].median(), df_events_1['quantity_sold(before_promo)'].mode()

(np.float64(138.43333333333334),
 np.float64(78.0),
 0    78.0
 Name: quantity_sold(before_promo), dtype: float64)

In [31]:
median_quantity_sold = df_events_1['quantity_sold(before_promo)'].median()
print(f"The median used to replace missing values is: {median_quantity_sold}")
median_quantity_sold

The median used to replace missing values is: 78.0


np.float64(78.0)

In [32]:
df_events_1['quantity_sold(before_promo)'].fillna(median_quantity_sold, inplace=True)

In [33]:
df_events_1['quantity_sold(before_promo)'].isnull().sum()

np.int64(0)

In [34]:
missing_values_after = df_events_1['quantity_sold(before_promo)'].isnull().sum()
print(f"Number of missing values after imputation: {missing_values_after}")

Number of missing values after imputation: 0


In [35]:
# Display the number of missing values filled
missing_values_filled = missing_values - missing_values_after
print(f"Number of missing values filled: {missing_values_filled}")


Number of missing values filled: 0


In [36]:
median_value = df_events_1['quantity_sold(before_promo)'].median()
median_value

np.float64(78.0)

#### Client Requests
*4. Identify the product category with the lowest base price before the promotion*

In [37]:
merged_df_products = pd.merge(df_events_1, df_products, on='product_code', how='inner')
merged_df_products.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),product_name,category
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Atliq_Scrub_Sponge_For_Dishwash,Home Care
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Atliq_Home_Essential_8_Product_Combo,Combo1


In [38]:
print(f"Number of rows and columns in merged dataframe: {merged_df_products.shape}")
merged_df_products.shape

Number of rows and columns in merged dataframe: (1500, 11)


(1500, 11)

In [39]:
df_events.shape, df_products.shape

((1500, 9), (15, 3))

In [40]:
min_base_price = merged_df_products['base_price(before_promo)'].min()
min_base_price

np.int64(50)

In [41]:
merged_df_products[merged_df_products['base_price(before_promo)'] == min_base_price]['category']

12      Personal Care
22      Personal Care
28      Personal Care
30      Personal Care
90      Personal Care
118     Personal Care
161     Personal Care
166     Personal Care
169     Personal Care
261     Personal Care
278     Personal Care
284     Personal Care
328     Personal Care
347     Personal Care
358     Personal Care
376     Personal Care
383     Personal Care
390     Personal Care
416     Personal Care
444     Personal Care
546     Personal Care
602     Personal Care
633     Personal Care
677     Personal Care
691     Personal Care
709     Personal Care
747     Personal Care
792     Personal Care
793     Personal Care
807     Personal Care
817     Personal Care
850     Personal Care
891     Personal Care
901     Personal Care
906     Personal Care
933     Personal Care
953     Personal Care
991     Personal Care
1067    Personal Care
1161    Personal Care
1179    Personal Care
1186    Personal Care
1191    Personal Care
1217    Personal Care
1223    Personal Care
1256    Pe

In [42]:
category_with_lowest_price = merged_df_products[merged_df_products['base_price(before_promo)'] == min_base_price]['category'].iloc[0]
category_with_lowest_price 

'Personal Care'

In [43]:
print(f"Product Category with lowest base price: {category_with_lowest_price}")

Product Category with lowest base price: Personal Care


#### Client Requests
*5. What is the total quantity sold after the promotion for the BOGOF promo type  during the Diwali campaign?*

In [44]:
df_events_1.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272


In [45]:
merged_df_products.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),product_name,category
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Atliq_Scrub_Sponge_For_Dishwash,Home Care
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Atliq_Home_Essential_8_Product_Combo,Combo1


In [46]:
merged_df_campaigns = pd.merge(df_events_1, df_campaigns, on='campaign_id', how='inner')
merged_df_campaigns.sample(3)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date
115,583386,STBLR-9,CAMP_SAN_01,P08,1190,63.0,BOGOF,595,254,Sankranti,10-01-2024,16-01-2024
1439,75f9b8,STCHE-4,CAMP_SAN_01,P06,415,27.0,25% OFF,311,22,Sankranti,10-01-2024,16-01-2024
570,1ee94b,STHYD-0,CAMP_DIW_01,P13,350,68.0,BOGOF,175,235,Diwali,12-11-2023,18-11-2023


In [47]:
# Filters the rows where both conditions (promo_type == 'BOGOF' and campaign_name == 'Diwali') are true and use .sum() method calculates the total for the selected values.
total_quantity_sold = merged_df_campaigns.loc[
                                        (merged_df_campaigns['promo_type'] == 'BOGOF') & 
                                        (merged_df_campaigns['campaign_name'] == 'Diwali'),
                                    'quantity_sold(after_promo)'].sum()

print(f"Total quantity sold after the promotion for BOGOF promo type during the Diwali campaign: {total_quantity_sold}")
total_quantity_sold

Total quantity sold after the promotion for BOGOF promo type during the Diwali campaign: 34461


np.int64(34461)

##### OR

In [48]:
# Step 2: Filter rows for BOGOF promo type and Diwali campaign
filtered_df = merged_df_campaigns[(merged_df_campaigns['promo_type'] == 'BOGOF') &
                                  (merged_df_campaigns['campaign_name'] == 'Diwali')]

# Step 3: Calculate the total quantity sold after the promotion
total_quantity_sold = filtered_df['quantity_sold(after_promo)'].sum()
print(f"Total quantity sold after the promotion for BOGOF promo type during the Diwali campaign: {total_quantity_sold}")


Total quantity sold after the promotion for BOGOF promo type during the Diwali campaign: 34461


#### Client Requests
*6. Which store recorded the highest quantity sold after the promotion during the Diwali campaign?*

In [49]:
merged_df_campaigns.sample(3)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date
591,6c9451,STVSK-0,CAMP_DIW_01,P03,156,259.0,25% OFF,117,225,Diwali,12-11-2023,18-11-2023
424,315542,STVJD-1,CAMP_SAN_01,P03,200,183.0,BOGOF,100,710,Sankranti,10-01-2024,16-01-2024
975,582098,STMYS-1,CAMP_SAN_01,P14,1020,109.0,BOGOF,510,453,Sankranti,10-01-2024,16-01-2024


In [50]:
# Filter rows for the Diwali campaign 
diwali_data = merged_df_campaigns[merged_df_campaigns['campaign_name'] == 'Diwali']
diwali_data.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Diwali,12-11-2023,18-11-2023


In [51]:
store_with_highest_sales = diwali_data.loc[diwali_data['quantity_sold(after_promo)'].idxmax(), 'store_id']
print(store_with_highest_sales)
df_stores[df_stores['store_id'] == store_with_highest_sales]

STCHE-4


,store_id,city
47,STCHE-4,Chennai


In [52]:
city_name = df_stores[df_stores['store_id'] == store_with_highest_sales].city.iloc[0]
print(city_name)

Chennai


In [53]:
highest_quantity_sold = diwali_data['quantity_sold(after_promo)'].max()
highest_quantity_sold

np.int64(1545)

In [54]:
print(f"The store with the highest quantity sold after the promotion during the Diwali campaign is Store ID: {store_with_highest_sales} and location of store is in {city_name}, with {highest_quantity_sold} units sold.")

The store with the highest quantity sold after the promotion during the Diwali campaign is Store ID: STCHE-4 and location of store is in Chennai, with 1545 units sold.


#### Client Requests
*7.  Understand which campaigns had the most successful outcomes. Compare the total quantities sold before and after the promotions for the Sankranti and Diwali campaigns. Which campaign saw a greater increase in sales?*

In [55]:
# Filter rows for Sankranti and Diwali campaigns
filtered_campaigns = merged_df_campaigns[merged_df_campaigns['campaign_name'].isin(['Sankranti', 'Diwali'])]
filtered_campaigns.head(3)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Diwali,12-11-2023,18-11-2023


In [56]:
sales_comparison = filtered_campaigns.groupby('campaign_name').agg(
    total_quantity_before = ('quantity_sold(before_promo)', 'sum'), 
    total_quantity_after = ('quantity_sold(after_promo)', 'sum')
)
sales_comparison.head()

,total_quantity_before,total_quantity_after
campaign_name,,
Diwali,109756.0,183404
Sankranti,97894.0,252069


In [57]:
sales_comparison['increase_in_sales'] = sales_comparison['total_quantity_after'] - sales_comparison['total_quantity_before']
sales_comparison

,total_quantity_before,total_quantity_after,increase_in_sales
campaign_name,,,
Diwali,109756.0,183404,73648.0
Sankranti,97894.0,252069,154175.0


In [58]:
# Determine which campaign saw a greater increase
greater_increase = sales_comparison['increase_in_sales'].idxmax()
increase_amount = sales_comparison.loc[greater_increase, 'increase_in_sales']
greater_increase, increase_amount

('Sankranti', np.float64(154175.0))

In [59]:
print(f"The campaign with the greater increase in sales is '{greater_increase}' with an increase of {increase_amount} units.")
print(sales_comparison)

The campaign with the greater increase in sales is 'Sankranti' with an increase of 154175.0 units.
               total_quantity_before  total_quantity_after  increase_in_sales
campaign_name                                                                
Diwali                      109756.0                183404            73648.0
Sankranti                    97894.0                252069           154175.0


#### Client Requests
*8. Which product recorded the highest Incremental Revenue Percentage (IR%) during the Sankranti campaign? What is the IR% for this product?*

IR% = ( (Revenue After Promo - Revenue Before Promo) / Revenue Before Promo ) * 100

Revenue = Base Price × Quantity Sold

In [60]:
# Merge fact_events with dim_campaigns and dim_products
events_with_campaigns = pd.merge(df_events_1, df_campaigns, on='campaign_id', how='left')
events_with_products = pd.merge(events_with_campaigns, df_products, on='product_code', how='left')

# Filter data for Sankranti campaign
sankranti_events = events_with_products[events_with_products['campaign_name'] == 'Sankranti'].copy()

# Calculate revenue before and after promo
sankranti_events['revenue_before_promo'] = sankranti_events['base_price(before_promo)'] * sankranti_events['quantity_sold(before_promo)']
sankranti_events['revenue_after_promo'] = sankranti_events['base_price(after_promo)'] * sankranti_events['quantity_sold(after_promo)']

# Group by product_name and sum the revenue columns
product_revenue = sankranti_events.groupby('product_name').agg(
    total_revenue_before_promo = ('revenue_before_promo', 'sum'), 
    total_revenue_after_promo = ('revenue_after_promo', 'sum')
)

product_revenue

,total_revenue_before_promo,total_revenue_after_promo
product_name,,
Atliq_Body_Milk_Nourishing_Lotion (120ML),239040.0,146462
Atliq_Cream_Beauty_Bathing_Soap (125GM),75400.0,42328
Atliq_Curtains,583500.0,1065450
Atliq_Doodh_Kesar_Body_Lotion (200ML),373350.0,258970
Atliq_Double_Bedsheet_set,2485910.0,4649330
Atliq_Farm_Chakki_Atta (1KG),6813550.0,12779800
Atliq_Fusion_Container_Set_of_3,561080.0,342722
Atliq_High_Glo_15W_LED_Bulb,1740550.0,3303125
Atliq_Home_Essential_8_Product_Combo,16185000.0,31027500


In [61]:
# Group by product_name and sum the revenue columns
product_revenue = sankranti_events.groupby('product_name').agg(
    total_revenue_before_promo = ('revenue_before_promo', 'sum'), 
    total_revenue_after_promo = ('revenue_after_promo', 'sum')
).reset_index()

product_revenue.head(2)

,product_name,total_revenue_before_promo,total_revenue_after_promo
0,Atliq_Body_Milk_Nourishing_Lotion (120ML),239040.0,146462
1,Atliq_Cream_Beauty_Bathing_Soap (125GM),75400.0,42328


In [62]:
sankranti_events.describe()

,base_price(before_promo),quantity_sold(before_promo),base_price(after_promo),quantity_sold(after_promo),revenue_before_promo,revenue_after_promo
count,750.000000,750.000000,750.000000,750.00000,750.000000,7.500000e+02
mean,554.933333,130.525333,365.866667,336.09200,76647.977333,1.169113e+05
std,740.836902,142.061078,601.114104,443.29151,117610.574738,1.796903e+05
min,50.000000,10.000000,31.000000,9.00000,550.000000,3.690000e+02
25%,90.000000,34.000000,67.000000,39.25000,4995.000000,3.283000e+03
50%,300.000000,60.000000,150.000000,166.00000,28884.000000,3.725750e+04
75%,860.000000,183.000000,510.000000,419.50000,81932.500000,1.510725e+05
max,3000.000000,642.000000,2500.000000,2067.00000,552120.000000,1.107500e+06


#### Client Requests
*9. Which store in Visakhapatnam recorded the lowest Incremental Sold Units Percentage (ISU%) during the Diwali campaign? What is the ISU% for that store?*

ISU% = (Quantity Sold After Promo - Quantity Sold Before Promo) / (Quantity Sold Before Promo) * 100

In [63]:
# Merge fact_events with dim_campaigns and dim_products
events_with_campaigns = pd.merge(df_events_1, df_campaigns, on='campaign_id', how='left')
events_with_products = pd.merge(events_with_campaigns, df_products, on='product_code', how='left')

# Filter data for Sankranti campaign
sankranti_events = events_with_products[events_with_products['campaign_name'] == 'Sankranti'].copy()

# Calculate revenue before and after promo
sankranti_events['revenue_before_promo'] = sankranti_events['base_price(before_promo)'] * sankranti_events['quantity_sold(before_promo)']
sankranti_events['revenue_after_promo'] = sankranti_events['base_price(after_promo)'] * sankranti_events['quantity_sold(after_promo)']

# Group by product_name and sum the revenue columns
product_revenue = sankranti_events.groupby('product_name').agg(
    total_revenue_before_promo = ('revenue_before_promo', 'sum'), 
    total_revenue_after_promo = ('revenue_after_promo', 'sum')
).reset_index()
print(product_revenue.shape)
product_revenue.head(3)

(15, 3)


,product_name,total_revenue_before_promo,total_revenue_after_promo
0,Atliq_Body_Milk_Nourishing_Lotion (120ML),239040.0,146462
1,Atliq_Cream_Beauty_Bathing_Soap (125GM),75400.0,42328
2,Atliq_Curtains,583500.0,1065450


In [71]:
events_with_products.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Diwali,12-11-2023,18-11-2023,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Diwali,12-11-2023,18-11-2023,Atliq_Scrub_Sponge_For_Dishwash,Home Care
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Sankranti,10-01-2024,16-01-2024,Atliq_Home_Essential_8_Product_Combo,Combo1


In [64]:
# Merge fact_events with dim_campaigns and dim_stores
events_with_stores = pd.merge(events_with_campaigns, df_stores, on='store_id', how='left')
# Filter data for Diwali campaign in Visakhapatnam
diwali_events = events_with_stores[(events_with_stores['campaign_name']=='Diwali') & (events_with_stores['city']=='Visakhapatnam')].copy()
print(diwali_events.shape)
diwali_events.head(2)

(75, 13)


,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,city
27,ba86f4,STVSK-1,CAMP_DIW_01,P13,350,61.0,BOGOF,175,204,Diwali,12-11-2023,18-11-2023,Visakhapatnam
64,9779b0,STVSK-2,CAMP_DIW_01,P10,65,85.0,50% OFF,32,128,Diwali,12-11-2023,18-11-2023,Visakhapatnam


In [72]:
events_with_stores.head(3)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Chennai
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Diwali,12-11-2023,18-11-2023,Bengaluru


In [65]:
# Group by store_id and sum the sold units
store_sold_units = diwali_events.groupby('store_id').agg(
    total_sold_units_before_promo = ('quantity_sold(before_promo)', 'sum'), 
    total_sold_units_after_promo = ('quantity_sold(after_promo)', 'sum') 	
).reset_index()
# store_sold_units
# Calculate Incremental Sold Units Percentage (ISU%)
store_sold_units['ISU%'] = (
    (store_sold_units['total_sold_units_after_promo'] - store_sold_units['total_sold_units_before_promo']) / (store_sold_units['total_sold_units_before_promo']) * 100
)
# store_sold_units
# Find the store with the lowest ISU%
lowest_isu_store = store_sold_units.loc[store_sold_units['ISU%'].idxmin()]

# Print the results
print(f"{lowest_isu_store['store_id']},{lowest_isu_store['ISU%']:.2f}")

STVSK-3,49.21


#### Client Requests
*10. Which promo type had both a negative Incremental Revenue Percentage (IR%) and Incremental Sold Units Percentage (ISU%) during the Sankranti campaign?*

In [66]:
sankranti_events.head(3)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,revenue_before_promo,revenue_after_promo
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,140230.0,300070
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Sankranti,10-01-2024,16-01-2024,Atliq_Home_Essential_8_Product_Combo,Combo1,366000.0,680000
7,6.88E+10,STVJD-0,CAMP_SAN_01,P08,1190,22.0,BOGOF,595,88,Sankranti,10-01-2024,16-01-2024,Atliq_Double_Bedsheet_set,Home Care,26180.0,52360


In [67]:
# Group by promo_type and sum the revenue columns
promo_revenue = sankranti_events.groupby('promo_type').agg(
    total_revenue_before_promo = ('revenue_before_promo', 'sum'), 
    total_revenue_after_promo = ('revenue_after_promo', 'sum'), 
    total_quantity_sold_before_promo = ('quantity_sold(before_promo)', 'sum'), 
    total_quantity_sold_after_promo = ('quantity_sold(after_promo)', 'sum')
).reset_index()

promo_revenue['IR%'] = (promo_revenue['total_revenue_after_promo'] - promo_revenue['total_revenue_before_promo']) / (promo_revenue['total_revenue_before_promo']) * 100
promo_revenue['ISU%'] = (promo_revenue['total_quantity_sold_after_promo'] - promo_revenue['total_quantity_sold_before_promo']) / (promo_revenue['total_quantity_sold_before_promo']) * 100
promo_revenue

,promo_type,total_revenue_before_promo,total_revenue_after_promo,total_quantity_sold_before_promo,total_quantity_sold_after_promo,IR%,ISU%
0,25% OFF,935195.0,567387,6601.0,5307,-39.329552,-19.603090
1,33% OFF,20483136.0,19273955,33624.0,47459,-5.903300,41.146205
2,50% OFF,527482.0,363564,4451.0,6100,-31.075563,37.047854
3,500 Cashback,16185000.0,31027500,5395.0,12411,91.705283,130.046339
4,BOGOF,19355170.0,36451055,47823.0,180792,88.327227,278.044037


In [68]:
negative_promo_types = promo_revenue[(promo_revenue['IR%'] < 0) & (promo_revenue['ISU%'] < 0)]
negative_promo_types

,promo_type,total_revenue_before_promo,total_revenue_after_promo,total_quantity_sold_before_promo,total_quantity_sold_after_promo,IR%,ISU%
0,25% OFF,935195.0,567387,6601.0,5307,-39.329552,-19.60309


In [69]:
print(f"{negative_promo_types.iloc[0]['promo_type']},{negative_promo_types.iloc[0]['IR%']:.2f},{negative_promo_types.iloc[0]['ISU%']:.2f}")

25% OFF,-39.33,-19.60


In [70]:
sankranti_events.to_csv('dim_sankranti_events.csv', index=False)